# Using the output of our luigi pipeline

In [24]:
import os
import dask.dataframe as dd
import datetime
import pandas as pd
import numpy as np

# The joined data transaction data

In [2]:
ENRICHED_TRXN_DATA = './DATA/PROCESSED/enriched_trxns/enriched_trxns-*.csv'

In [3]:
!ls -lsrt $ENRICHED_TRXN_DATA | head -5

40 -rw-r--r--  1 evanwelch  staff  19986 Mar 14 23:37 ./DATA/PROCESSED/enriched_trxns/enriched_trxns-32.csv
48 -rw-r--r--  1 evanwelch  staff  21867 Mar 14 23:37 ./DATA/PROCESSED/enriched_trxns/enriched_trxns-07.csv
40 -rw-r--r--  1 evanwelch  staff  19240 Mar 14 23:37 ./DATA/PROCESSED/enriched_trxns/enriched_trxns-00.csv
48 -rw-r--r--  1 evanwelch  staff  20600 Mar 14 23:37 ./DATA/PROCESSED/enriched_trxns/enriched_trxns-15.csv
48 -rw-r--r--  1 evanwelch  staff  22145 Mar 14 23:37 ./DATA/PROCESSED/enriched_trxns/enriched_trxns-03.csv


In [4]:
! cat ./DATA/PROCESSED/enriched_trxns/enriched_trxns-32.csv | head -1

_id|amount|cust_first_nm|cust_last_nm|cust_state|cust_zip|customer_id|medium|merchant_id|mrch_catgs|mrch_lat|mrch_lon|mrch_name|mrch_state|mrch_zip|payer_id|purchase_date|rewards|status|type


In [5]:
! cat ./DATA/PROCESSED/enriched_trxns/enriched_trxns-32.csv | head -2 | tail -1

5aa9edfff0cec56abfa3f178|314.73842349112346|Michelle|Sawyer|IA|08636|5aa9eacef0cec56abfa3deac|balance|593848ceceb8abe242510b33|['travel']|0.0|0.0|priceline|CA|90503|5aa9ebabf0cec56abfa3e3c5|2017-12-01|0|cancelled|merchant


# Read it in for exploration...

In [7]:
# using dask dataframes, very similar to pandas
ddf = dd.read_csv(ENRICHED_TRXN_DATA, sep='|', dtype={'mrch_zip':str, 'zip': str})

In [8]:
ddf.head()

,_id,amount,cust_first_nm,cust_last_nm,cust_state,cust_zip,customer_id,medium,merchant_id,mrch_catgs,mrch_lat,mrch_lon,mrch_name,mrch_state,mrch_zip,payer_id,purchase_date,rewards,status,type
0,5aa9ec91f0cec56abfa3e91a,25.207649,Kimberly,Patterson,PA,16041,5aa9ea97f0cec56abfa3dd6c,balance,58a8de4c1756fc834d9053c6,['Shopping'],39.465186,-76.641839,Home Depot,MD,21030,5aa9eb44f0cec56abfa3e154,2017-12-01,0,cancelled,merchant
1,5aa9ec91f0cec56abfa3e91b,261.483298,Kimberly,Patterson,PA,16041,5aa9ea97f0cec56abfa3dd6c,balance,57cf75cfa73e494d8675f80b,"['gym', 'lodging', 'health', 'restaurant', 'fo...",38.913722,-77.221603,Tysons Corner Marriott,VA,22182,5aa9eb44f0cec56abfa3e155,2017-12-02,0,cancelled,merchant
2,5aa9ec91f0cec56abfa3e91c,549.429422,Kimberly,Patterson,PA,16041,5aa9ea97f0cec56abfa3dd6c,balance,59384a53ceb8abe24251227f,['travel'],0.000000,0.000000,priceline,CA,90503,5aa9eb44f0cec56abfa3e155,2017-12-04,0,cancelled,merchant
3,5aa9ec91f0cec56abfa3e91d,25.811945,Kimberly,Patterson,PA,16041,5aa9ea97f0cec56abfa3dd6c,balance,57cf75cfa73e494d8675f766,"['shopping_mall', 'health', 'store', 'point_of...",30.623971,-96.304156,Proactiv Kiosk,TX,77840,5aa9eb44f0cec56abfa3e155,2017-12-05,0,cancelled,merchant
4,5aa9ec91f0cec56abfa3e91e,28.053279,Kimberly,Patterson,PA,16041,5aa9ea97f0cec56abfa3dd6c,balance,57cf75cea73e494d8675f0a2,"['shopping_mall', 'department_store', 'jewelry...",42.461314,-71.028013,Square One Mall,MA,01906,5aa9eb44f0cec56abfa3e155,2017-12-07,0,cancelled,merchant


# Ok, so say we want to make some features.... 
* total, average, and count ...
* of credit card transactions...
* per cusomter
* for certain categories

# one way do it...

In [11]:
ddf[ddf.purchase_date == '2017-12-01'].dropna().groupby('customer_id').amount.sum().compute().head()

customer_id
5aa9ea97f0cec56abfa3dd6c     25.207649
5aa9ea97f0cec56abfa3dd6d     26.857885
5aa9ea97f0cec56abfa3dd6e     24.509473
5aa9ea97f0cec56abfa3dd71     29.437342
5aa9ea98f0cec56abfa3dd72    104.566699
Name: amount, dtype: float64

# A longer, but more robust way ....

## filter certain rows (in or out)

In [42]:
def no_rewards_purchases(trxn):
    return trxn['medium'] != 'reward'

def only_this_catg(trxn, catg):
    return any([catg.lower() in cat.lower() for cat in trxn['mrch_catgs'].split()])

def only_this_date(trxn, year=None, month=None, day=None):
    result = True
    try:
        trxn_dt = pd.to_datetime(trxn['purchase_date'])
    except:
        trxn_dt = pd.NaT
    if year:
        result &= trxn_dt.year == year
    if month:
        result &= trxn_dt.month == month
    if day:
        result &= trxn_dt.day == day
    
    return result



## some new columns

In [26]:
def first_digit_zip_code(zip_str):
    if len(zip_str) >= 5:
        return int(zip_str[0])
    else:
        return np.nan

In [27]:
def is_amazon(trxn):
    return 'amazon' in trxn['mrch_name'].lower()

## some aggregations

In [47]:
def _trxn_cnt(df):
    return df.shape[0]

def _trxn_total(df):
    return df['amount'].sum()

def _trxn_mean(df):
    return df['amount'].mean()

agg_operations = {
    'cnt': _trxn_cnt,
    'tot': _trxn_total,
    'avg': _trxn_mean}


In [29]:
# where to save features

FEATURE_PATH = 'DATA/FEATURES/'

if not os.path.exists(FEATURE_PATH):
    os.makedirs(FEATURE_PATH)

In [60]:
def write_feature(srs, feat_path):
    feature_folder = os.path.join(FEATURE_PATH, feat_path)
    if not os.path.exists(feature_folder):
        os.makedirs(feature_folder)
    
    # write the feature to disk
    files = srs.to_csv(os.path.join(FEATURE_PATH, feat_path, 'part-*.csv'), sep='|', index_label='customer_id')
    return files

In [61]:
catgories_of_interest = ['grocery', 'food', 'bar']
dates = pd.date_range('2017-12-01', '2017-12-10')

for catg in catgories_of_interest:
    for dt in dates:

        # apply filters
        keep_rows = (
                ddf.apply(no_rewards_purchases, axis=1) & 
                ddf.apply(lambda row: only_this_catg(row, catg=catg), axis=1) &
                ddf.apply(lambda row: only_this_date(row, year=dt.year, month=dt.month, day=dt.day), axis=1))
    
        # prepare for aggregate functions
        g = ddf.loc[keep_rows, :].groupby('customer_id')
        
        feat_path_template = '{dt}_{catg}_{op}'
        
        for agg_name, agg_func in agg_operations.items():
            feat_path = feat_path_template.format(
                dt=dt.strftime('%Y%m%d'),
                catg=catg,
                op=agg_name)
        
            # write the feature to disk
            write_feature(g.apply(agg_func), feat_path)
        

/Users/evanwelch/PROJECTS/production-feature-engineering/env/lib/python3.6/site-packages/dask/dataframe/core.py:2779: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)
/Users/evanwelch/PROJECTS/production-feature-engineering/env/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result


KeyboardInterrupt: 

In [ ]:
! ls DATA/FEATURE